## **Replica Exchange Monte Carlo simulation of three dimensional hard-spheres system** ##

### Introduction

With the great advancement of computational science over the past decades, partly due to the rapid growth of computer technologies, scientists in many research fields are now attacking realistic problems with large degrees of freedom and complexity. When dealing with systems with rough energy landscape, simulations by conventional molecular dynamics (MD) or Monte Carlo (MC) methods are of little use, because at low temperatures they tend to get trapped in local-minimum energy states and sample only a very limited region of the configurational space. The development of new powerful simulation algorithms that can alleviate this difficulty is thus particularly important. One powerful algorithm is the replica-exchange method (REM).  In this method, a number of non-interacting copies of the original system at different temperatures are simulated independently and simultaneously by the conventional MD or MC methods. Every few steps, pairs of replicas are exchanged with a specified transition probability. This exchange process realizes a random walk in temperature space, which in turn induces a random walk in the energy space so that a wide configurational space can be sampled during the simulation. In this work, we tests the performance of replica-exchange method. This report is organized as follows. Set up section provides the parameters and functions embedded in the system with source code definition. Other sections shows results and discussion of REMC and non-exchange MC simulations. 

In [1]:
from IPython.display import IFrame
IFrame(src='system.html', width=600, height=600)

<img src="system.png" style="width:300px;height:300px"/>

### Set up

In this work, I apply the replica exchange MC method to a three dimensional hard-spheres Lennard-Jones system. The potential energy between a pair of particle i and j, separated by a distance $r_{ij}$, is defined by the following function, where $\epsilon$ is the depth of the attractive well, and $\sigma$ is a parameter that controls the size of the particles. In order to mimic bulk particle system interactions, periodic boundary conditions are employed (see above plot). The Lennard-Jones potential-energy function is truncated at a cutoff distance $r_c$.

$U_{i j}=4 \epsilon\left[\left(\frac{\sigma}{r_{i j}}\right)^{12}-\left(\frac{\sigma}{r_{i j}}\right)^{6}\right] \space $ 

#### _Atom and configuration definition_

In [5]:
class Atom:
    x = y = z = 0.0
    mass = 0.0 
    radius = 0.0
    energy = 0.0
class Configuration:
    def __init__(self,na):
        self.natoms = na
        self.COM = [0.0,0.0,0.0]
        self.atom = []
        for i in range(na):
            self.atom.append(Atom())
    #center of mass calculation
    def CalcCOM(self):     
        M = 0.0
        sumx = sumy = sumz = 0.0
        for i in range(0, self.natoms):
            m = self.atom[i].mass
            sumx += self.atom[i].x * m
            sumy += self.atom[i].y * m
            sumz += self.atom[i].z * m
            M += m

        Cx = sumx/M
        Cy = sumy/M
        Cz = sumz/M
        self.COM[0]=Cx
        self.COM[1]=Cy
        self.COM[2]=Cz
    #Radius of gyration calculation
    def RadGyr(self):   
        sumgyr = 0.0
        M = 0.0
        self.CalcCOM()
        comx = self.COM[0]
        comy = self.COM[1]
        comz = self.COM[2]
        for i in range(0, self.natoms):
            M += self.atom[i].mass
            mc = self.atom[i].mass
            rx = self.atom[i].x
            ry = self.atom[i].y
            rz = self.atom[i].z
            sgx = (rx - comx)**2.0
            sgy = (ry - comy)**2.0
            sgz = (rz - comz)**2.0

            sumgyr += mc*(sgx+sgy+sgz)

        Rsq = sumgyr / M
        R = math.sqrt(Rsq)
        return R

#### _System parameters definition_

In [ ]:
N = 10  #number of partical
L = 15 #length of box
d = L/N  #diameter of hard-sphere
steps = 10000  #total steps
sigma=1.5*d   #sigma in lj potential
epsilon=2*d  #epsilon in lj potential
temp=[300.00,362.00,436.00,520.00,600.00,670.00,750.00,830.00] #temperature k
kb=1.38e-23*6.022e23/(1000)  #bolzmann constant kJ/mol
cutoff=L/2-1  #cutoff of pair-wise interaction
max_move=0.5  #maximum move 
mass = 12.0 #mass of partical
center = [L/2,L/2,L/2] #center of box

#### _Useful functions definition_

In [ ]:
#check if inbox
def inbox(x,y,z):     
    if x > L-d/2:
        x -= L-d
    elif x < d/2:
        x += L - d

    if y > L-d/2:
        y -= L-d
    elif y < d/2:
        y += L - d
  
    if z > L-d/2:
        z -= L-d
    elif z < d/2:
        z += L - d   
    return x,y,z
#lj potential
def lj_potential(r):
    energy = 4*epsilon*((sigma/r)**12-(sigma/r)**6)
    return energy
#pair-wise interaction calculation
def energy(r,x,y,z,n,compute_all):
    '''compute total energy of system or one particle energy'''
    if compute_all:
        r[n][0],r[n][1],r[n][2]=x,y,z
    x_left = list(r[:,0]-L)
    x_right = list(r[:,0]+L)
    y_left = list(r[:,1]-L)
    y_right = list(r[:,1]+L)
    z_left = list(r[:,2]-L)
    z_right = list(r[:,2]+L)
    all_box = np.array([(x_left*3+list(r[:,0])*3+x_right*3)*3, \
                        ((y_left+list(r[:,1])+y_right)*3)*3, \
                        z_right*9+list(r[:,2])*9+z_left*9])
    all_box = all_box.transpose((1,0))
    i = 13*N
    distance = []
    if compute_all: 
        while i < 14*N:
            distance += [np.sqrt((all_box[i][0]-all_box[n][0])**2+ \
                                 (all_box[i][1]-all_box[n][1])**2+ \
                                 (all_box[i][2]-all_box[n][2])**2) for n in \
                         list(np.linspace(0,13*N-1,13*N,dtype=int))+ \
                         list(np.linspace(i+1,27*N-1,27*N-i-1,dtype=int))]
            i += 1
        energy = [lj_potential(d) for d in distance if d < cutoff and d > 0]
        energy_sum = sum(energy)
    else:      
        r0 = np.array([[x for d in range(27*N)],[y for d in range(27*N)],\
                       [z for d in range(27*N)]]) 
        a = r0.transpose((1,0)) - all_box  #compare the updated particle coordinate with others
        distance = [np.sqrt(a[i][0]**2+a[i][1]**2+a[i][2]**2) for i in range(27*N)]  #calculate distance     
        Energy = [lj_potential(d) for d in distance if d < cutoff and d > 0]
        energy_sum = sum(Energy)
    return energy_sum
#metropolis move criterion
def move(r, x,y,z,n,e, diff_e, temp,accepted_step):                         
    '''Metropolis Monte carlo move criterion'''
    if diff_e < 0.0:
        e = e_new
        r[n][0],r[n][1],r[n][2] = inbox(x_new,y_new,z_new)
        accepted_step+=1
    else:
        rand = random.uniform(0,1)
        if math.exp(-diff_e/(kb*temp)) > rand:
            e = e_new
            accepted_step+=1
            r[n][0],r[n][1],r[n][2] = inbox(x_new,y_new,z_new)
    return r,e,accepted_step

#### _Determine how to exchange_

In [ ]:
def delta(temp1,temp2,energy1,energy2):
    '''delta calculation'''
    ddelta = (1/(kb*temp1) - 1/(kb*temp2))*(energy2-energy1)
    return ddelta
def exchange(frame,rg,energy,numexchg):
    '''for eight replicas, we try to exchange different 
       adjacent replicas at odd and even enchange trials 
       following metropolis criterion.'''
    exchg_order=[[[0,1],[2,3],[4,5],[6,7]],[[0,7],[1,2],[3,4],[5,6]]]
    if numexchg % 2 == 0:
        exchg_order = exchg_order[0]
    else:
        exchg_order = exchg_order[1]
    for i in exchg_order:
        dd = delta(temp[i[0]],temp[i[1]],energy[i[0]][(numexchg+1)*(steps+1)-1],energy[i[1]][(numexchg+1)*(steps+1)-1])
        if dd <= 0:
            frame[i[0]][(numexchg+1)*(steps+1)] = frame[i[1]][(numexchg+1)*(steps+1)-1]
            frame[i[1]][(numexchg+1)*(steps+1)] = frame[i[0]][(numexchg+1)*(steps+1)-1]
            energy[i[0]][(numexchg+1)*(steps+1)] = energy[i[1]][(numexchg+1)*(steps+1)-1]
            energy[i[1]][(numexchg+1)*(steps+1)] = energy[i[0]][(numexchg+1)*(steps+1)-1]
            rg[i[0]][(numexchg+1)*(steps+1)] = rg[i[1]][(numexchg+1)*(steps+1)-1]
            rg[i[1]][(numexchg+1)*(steps+1)] = rg[i[0]][(numexchg+1)*(steps+1)-1]
            temp_exchange[numexchg+1][i[0]] = temp_exchange[numexchg][i[1]]
            temp_exchange[numexchg+1][i[1]] = temp_exchange[numexchg][i[0]]
            accepted_exchange[numexchg][i[0]] = 1
            accepted_exchange[numexchg][i[1]] = 1

            accepted_exchange_ratio[numexchg][i[0]] = accepted_exchange[0:numexchg+1,i[0]].sum()/(numexchg+1)
            accepted_exchange_ratio[numexchg][i[1]] = accepted_exchange[0:numexchg+1,i[1]].sum()/(numexchg+1)
        else:
            rand = random.uniform(0,1)
            if math.exp(-dd) > rand:
                frame[i[0]][(numexchg+1)*(steps+1)] = frame[i[1]][(numexchg+1)*(steps+1)-1]
                frame[i[1]][(numexchg+1)*(steps+1)] = frame[i[0]][(numexchg+1)*(steps+1)-1]
                energy[i[0]][(numexchg+1)*(steps+1)] = energy[i[1]][(numexchg+1)*(steps+1)-1]
                energy[i[1]][(numexchg+1)*(steps+1)] = energy[i[0]][(numexchg+1)*(steps+1)-1]
                rg[i[0]][(numexchg+1)*(steps+1)] = rg[i[1]][(numexchg+1)*(steps+1)-1]
                rg[i[1]][(numexchg+1)*(steps+1)] = rg[i[0]][(numexchg+1)*(steps+1)-1]
                temp_exchange[numexchg+1][i[0]] = temp_exchange[numexchg][i[1]]
                temp_exchange[numexchg+1][i[1]] = temp_exchange[numexchg][i[0]]
                accepted_exchange[numexchg][i[0]] = 1
                accepted_exchange[numexchg][i[1]] = 1
                accepted_exchange_ratio[numexchg][i[0]] = accepted_exchange[0:numexchg+1,i[0]].sum()/(numexchg+1)
                accepted_exchange_ratio[numexchg][i[1]] = accepted_exchange[0:numexchg+1,i[1]].sum()/(numexchg+1)
            else:
                frame[i[0]][(numexchg+1)*(steps+1)] = frame[i[0]][(numexchg+1)*(steps+1)-1]
                frame[i[1]][(numexchg+1)*(steps+1)] = frame[i[1]][(numexchg+1)*(steps+1)-1]
                energy[i[0]][(numexchg+1)*(steps+1)] = energy[i[0]][(numexchg+1)*(steps+1)-1]
                energy[i[1]][(numexchg+1)*(steps+1)] = energy[i[1]][(numexchg+1)*(steps+1)-1]
                rg[i[0]][(numexchg+1)*(steps+1)] = rg[i[0]][(numexchg+1)*(steps+1)-1]
                rg[i[1]][(numexchg+1)*(steps+1)] = rg[i[1]][(numexchg+1)*(steps+1)-1]
                temp_exchange[numexchg+1][i[0]] = temp_exchange[numexchg][i[0]]
                temp_exchange[numexchg+1][i[1]] = temp_exchange[numexchg][i[1]]
                accepted_exchange[numexchg][i[0]] = 0
                accepted_exchange[numexchg][i[1]] = 0
                accepted_exchange_ratio[numexchg][i[0]] = accepted_exchange[0:numexchg+1,i[0]].sum()/(numexchg+1)
                accepted_exchange_ratio[numexchg][i[1]] = accepted_exchange[0:numexchg+1,i[1]].sum()/(numexchg+1)

#### _system initialization_

In [ ]:
c = Configuration(N) 
#initial random coordinates
for index in range(N):
    c.atom[index].x = center[0]+ (L-d) * (random.random()-0.5)
    c.atom[index].y = center[1]+ (L-d) * (random.random()-0.5)
    c.atom[index].z = center[2]+ (L-d) * (random.random()-0.5)
    c.atom[index].mass = mass
    c.atom[index].radius = d/2
point = np.array([[c.atom[i].x for i in range(N)],[c.atom[i].y for i in range(N)],
                  [c.atom[i].z for i in range(N)]])
point = point.transpose((1,0))
e = energy(point,0,0,0,0,compute_all=True)
#output matrix 
all_energy = np.zeros(steps+1)
all_energy[0]=e
all_frame = np.zeros((steps+1,N,3))
all_frame[0] = point
all_accepted_ratio=[]
accepted_step=0
average_energy= np.zeros(steps+1)
average_energy[0] = e
RG = np.zeros(steps+1)
RG[0] = c.RadGyr()
all_particle_energy = np.zeros((steps,N))

#### _running replica exchange Monte carlo simulation_

In [ ]:
for n in range(numexchg):
    for t in range(len(temp)):
        x_init = all_frame[t][n*(steps+1)][:,0]
        y_init = all_frame[t][n*(steps+1)][:,1]
        z_init = all_frame[t][n*(steps+1)][:,2]
        e = all_energy[t][n*(steps+1)]
        point = np.array([x_init,y_init,z_init])
        point = point.transpose((1,0))
        for i in range(steps):
            for m in range(N): 
                x,y,z = point[m][0],point[m][1],point[m][2]
                x_new = x+max_move*random.uniform(-1,1)
                y_new = y+max_move*random.uniform(-1,1)
                z_new = z+max_move*random.uniform(-1,1)
                e_new = energy(point,x_new,y_new,z_new,m,compute_all=True)
                diff_e = e_new - e
                point, e = move(point,x_new,y_new,z_new,m,e,diff_e,temp[t])
                c.atom[m].x,c.atom[m].y,c.atom[m].z = point[m][0],point[m][1],point[m][2]          
            RG[t][i+1+(steps+1)*n] = c.RadGyr() 
            all_energy[t][i+1+(steps+1)*n] = e
            all_frame[t][i+1+(steps+1)*n] = point      
            average_energy[t][i+1+(steps+1)*n] = all_energy[t][0:i+2+(steps+1)*n].sum()/(i+2+(steps+1)*n)
    exchange(all_frame,RG,all_energy,n)

### Results 

#### 1. Non-exchange Monte Carlo

In the beginning, I tried to run 500000 steps for a system with 50 hard-spheres under 300 K, however, it would take more than one day to finish due to the low efficiency of my code. Then, I tried to run 50000 steps for a smaller system with 10 hard-spheres under 300 K. The total energy and average energy of system are showed in the following figure, from which we can see it has not completely converged and proper treatment such as doing minimization before simulation and running longer equilibration may be helpful.
<img src="std_energy.png" style="width:600px;height:300px"/>

#### 2. Acceptance rates vs T/epsilon pairs

A set of MC simulations without exchange were done with different temperatures and epsilons. From the results showed in the following table, we can see the trend that the acceptance rates decrease with the increase of epsilon and decrease of temperature. It reflects that hiigher temperature would help to avoid getting trapped in the local minima along energy surface. This trend is not perfectly followed, which may be because more steps are needed to make sure all simulations are converged. (Each T/epsilon pair is run for only 5000 steps) 

| Temperature/Epsilon | 0.75 | 1.50 | 3.0 | 7.50 |
| --- | --- | --- | --- | --- |
| 200.00 | 0.58 | 0.49 | 0.31 | 0.02 |
| 246.00 | 0.58 | 0.57 | 0.35 | 0.05 |
| 300.00 | 0.58 | 0.57 | 0.46 | 0.18 |
| 362.00 | 0.72 | 0.65 | 0.46 | 0.10 |
| 436.00 | 0.64 | 0.52 | 0.43 | 0.20 |
| 520.00 | 0.67 | 0.61 | 0.42 | 0.26 |


#### 3. Temperature exchange 

In this work, we had eight replicas with same starting conformation under different temperature. Each replica follows the routine with same non-exchange monte carlo steps and one exchange trial. The temperature exchange is showed in the following figure, from which we can see exchange between two replicas were accepted many times. The following table showed the exchange accepted ratio between nearby replicas. Two things should be pointed out: 1) the accepted ratio from walker_1 to walker_7 are equally high, which is expected to happen and this also shows the choosed temperatures are helpful for exchange. 2) the accepted ratio of walker_1 and walker_8 are lower because they tried to exchange with each other along simulation, which showed in the above code, but exchange between these two replicas are hardly accepted because the energy difference is relatively large. <img src="plot_temp_ex_dataframe_100.png" style="width:400px;height:300px"/>

| | Walker_1 | Walker_2 | Walker_3 | Walker_4 | Walker_5 | Walker_6 | Walker_7 | Walker_8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Exchange accepted ratio | 0.76 | 0.91 | 0.91 | 0.91 | 0.94 | 0.95 | 0.93 | 0.77 |

#### 4. REMC energy comparison with non-exchange MC

Energy calculation is important for further study with other thermodynamic properties. The key advantage of replica exchange method is to sample broadly on energy surface under high temperature, which can be seen in the following figure. Each line represents the energy value under each temperature. The total energy shows that REMC can sample much more on energy surface compared with non-exchange MC in section 1. under 300 K. It also reflects the fact that the average energy along simulation is lower under lower temperature.

<img src="./remd_100_energy.png" style="width:600px;height:300px"/>

In order to check the performance of REMC, the energy is compared with non-exchange MC simulations under the same set of temperatures. We can see non-exchange MC showed similar behavior with different temperture and the avrerage energy compared in the following figure showed the performance of REMC except the replica under 520 K, which doesn't match the average energy value in non-exchange MC.

<img src="non_exchg_energy.png" style="width:900px;height:300px"/>

#### 5. Radius of gyration

A radius of gyration in general is the distance from the center of mass of a body at which the whole mass could be concentrated without changing its moment of rotational inertia about an axis through the center of mass. The calculation is wrapped as a function in configuration class showed in the above code. The following plot illustrates the comparison of average radius of gyration between REMC and on-exchange MC.

<img src="rg_compare.png" style="width:300px;height:300px"/>

### Discussion

In this report, we can see the comparison of performance between replica exchange method and conventional Monte Carlo simulation. Due to time limit, non-exchange MC has not completely converged after 50000 steps metropolis move. Simulations by taking advantage of replica exchange showed expected performance on configurational space sampling and avoiding getting trapped in local minimum energy states with only 10000 steps in total for each replica (100 metropolis move and 100 exchange trials). The high accepted exchange ratio may be caused by the simiplicity of system, which makes the energy difference low between each two replicas and thus increases the accepted exchange ratio. In more complicated systems like protein, the accepted exchange ratio are expected to be much lower. 

#### Note
Some researches point out more exchange trials would make replica exchange method more efficient. Similar calculations (50 metropolis move and 200 exchange trials;100 metropolis move and 100 exchange trials; 200 metropolis move and 50 exchange trials) are done, but the simulation has not finished and results will be analyzed later. 
The low efficiency of my code comes from energy calculation because after each atomic metropolis move, it will calculate all interactions in the system, which should be achieved by only calculating interaction around the atom after its move. However, I did in the above cheaper way before, but the output of atomic energy is as low as -100000, which is clearly wrong by analytically calculating of defined L-J potentials of single atom. Therefore, I had to use the much more expensive way and it will be fixed later. 